Updating stations in OSCAR/Surface
---
> **Example #3:** We will update the schedules of German stations. We want to update stations that only have a _30 min reporting interval_ . Their reporting interval  should be changed to 1h and the international exchange flag set to _true_. In case there is no information on start and endtime of the schedule, set it to correspond to a 7/24 schedule.

***Note***: we limit ourselves to observations tracked by WDMQS for the purpose of this exercise. We thus look for **ongoing** observations of _pressure, temperature, wind_ and _humidity_ at stations.




In [1]:
import json
import sys
import logging
logging.basicConfig(level=logging.INFO)

from oscar_lib import OscarClient, extractSchedules, Station

wdqms_variables = [216,210,224,251,309,12005,12006] # pressure,precipitation,temperature,humidity,wind deprecated, wind speed, wind direction

INFO:root:loading schema files
INFO:root:schema parsed sucessfully
INFO:root:schema parsed sucessfully
INFO:root:loading XSLT files
INFO:root:XSLT parsed sucessfully
INFO:root:XSLT simple2wmdr parsed sucessfully


In [3]:
client = OscarClient(oscar_url = OscarClient.OSCAR_DEPL, token="248eadae-8d38-4410-85c9-ccfd0ba897fa")
stations=client.oscarSearch({'territoryName':'DEU', 'facilityType':'landFixed'})

INFO:oscar_lib.oscar_client:searching for {'territoryName': 'DEU', 'facilityType': 'landFixed'} at https://oscardepl.wmo.int/surface//rest/api/search/station


In [4]:
# print out the first search result
stations["data"][0] 

{'id': 12494,
 'name': 'FILCHNER',
 'region': 'Antarctica',
 'territory': 'Germany',
 'declaredStatus': 'Operational',
 'latitude': -80.43681,
 'longitude': -44.43199,
 'elevation': 103,
 'stationTypeName': 'Land (fixed)',
 'wigosStationIdentifiers': [{'wigosStationIdentifier': '0-20000-0-89047',
   'primary': True}],
 'stationTypeId': 1,
 'dateEstablished': '2016-01-14T23:00:00.000+0000',
 'stationStatusCode': 'operational',
 'stationTypeCode': 'landOceanSurface',
 'stationProgramsDeclaredStatuses': 'GOS:Operational'}

In [5]:
# collect primary wigos ids from search result. 
# Need to loop through search result and potential multiple wigos ids to find the primary one
german_station_wigosids = [ wid["wigosStationIdentifier"] for station in stations["data"] for wid in station["wigosStationIdentifiers"] if wid["primary"] ]

#print out first 5 entries
german_station_wigosids[0:5]

['0-20000-0-89047',
 '0-20000-0-89003',
 '0-20000-0-89002',
 '0-20008-0-NMY',
 '0-20000-0-89011']

In [6]:
limit=10
station_map = {}
for wigos_id in german_station_wigosids[0:min(limit,len(german_station_wigosids))]:
    try:
        myxml = client.load_station(wigos_id=wigos_id)
        s = Station(myxml)
        
        if "20008" in wigos_id: # ignore GAWSIS stations
            continue
        
        # check if the station has at least one ongoing 30 min schedule (for WDQMS relevant variables)
        schedules = s.current_schedules(variables=wdqms_variables)

        # loop through nested datastructure and check if interval is 30min
        intervals = [ dg["schedule"]["interval"] == "PT30M" for o in schedules.values() for d in o["deployments"] for dg in d["datagenerations"] ]

        if any(intervals): # we consider the station as long as there is one 30min schedule
            station_map[wigos_id]=s
    
    except Exception as e:
        print("error loading station {} {}".format(wigos_id,str(e)))


INFO:root:input not JSON.. try XML in WMDR
INFO:root:station initialized


init


INFO:root:input not JSON.. try XML in WMDR
INFO:root:station initialized


init


INFO:root:input not JSON.. try XML in WMDR
INFO:root:station initialized


...

init


INFO:root:input not JSON.. try XML in WMDR
INFO:root:station initialized


init


INFO:root:input not JSON.. try XML in WMDR
INFO:root:station initialized


init


Output a schedule of a station. 251 stands for [humidity](https://codes.wmo.int/wmdr/ObservedVariableAtmosphere/_251)

In [7]:
# output a schedule of a station. 251 is 
print(json.dumps(list(station_map.values())[0].current_schedules()[251],indent=1))

{
 "gid": "id_261cc340-9621-48d8-908c-14a6fe30480f",
 "variableid": "http://codes.wmo.int/wmdr/ObservedVariableAtmosphere/251",
 "deployments": [
  {
   "gid": "id_4d842e96-a8e9-4940-bf72-f02d62a2c3f8",
   "from": "2016-01-12T00:00:00Z",
   "to": null,
   "datagenerations": [
    {
     "gid": "id_a1d9793a-d1b7-404e-8dcb-56d7b636a5c9",
     "schedule": {
      "from": "2016-01-12T00:00:00Z",
      "to": null,
      "startMonth": 1,
      "endMonth": 12,
      "startWeekday": 1,
      "endWeekday": 7,
      "startHour": 20,
      "endHour": 19,
      "startMinute": 0,
      "endMinute": 59,
      "interval": "PT30M",
      "international": true
     }
    }
   ]
  }
 ]
}


### update strategy: 
 1. set 1h international 7/24 schedule for "ongoing" schedules. 
 2. other schedules will be merged with the same 1h international 7/24 schedule, preserving the elements present in the original schedule



In [8]:
defaultSchedule = { 
    "from": "2016-04-28T00:00:00Z", "to" : None,     
    "startMonth": 1,"endMonth": 12,
    "startWeekday": 1, "endWeekday": 7,
    "startHour": 0, "startMinute": 0,          
    "endHour": 23,    "endMinute": 59,
    "interval": "PT1H",     "international": True   
}


In [9]:
for wigos_id,station in station_map.items():
    print("updating station",wigos_id)
    
    current_schedules = station.current_schedules() # get "ongoing" schedules
    for current_schedule in current_schedules.values():
        for d in current_schedule["deployments"]:
            for dg in d["datagenerations"]:
                gid=dg["gid"]
                schedule=dg["schedule"]
                
                station.fix_and_update_datageneration(gid,defaultSchedule) 

    try:
        station.validate()
        print("validates ok")
    except: # if there are remaining validation issues
        print("fixing remaining schedules")
        station.fix_deployments(mode="update",defaultSchedule=defaultSchedule) #merge remaining eronous schedules

    station.validate()
    
    
    station.save(client)
    print("saved",wigos_id)
    

updating station 0-20000-0-10505
validates ok
saved 0-20000-0-10505
updating station 0-20000-0-10309
validates ok
saved 0-20000-0-10309
updating station 0-20000-0-10442
validates ok
saved 0-20000-0-10442
updating station 0-20000-0-10520
validates ok
saved 0-20000-0-10520
